In [ ]:
!pip install neo4j-driver

In [ ]:
from neo4j import GraphDatabase, basic_auth
import pandas as pd

In [ ]:
driver = GraphDatabase.driver( "bolt://34.232.210.219:7687", auth=basic_auth("neo4j", "helicopters-probes-boost"))
def read_cypher(query):
  with driver.session(database="neo4j") as session:
    results = session.run(query).data()
    df = pd.DataFrame(results)
    col_names = df.columns.values.tolist()
    for c in col_names:
        if type(df[c].iloc[0]) is dict:
          df = pd.concat([df, df[c].apply(pd.Series)], axis=1)
          df.drop(columns=c, inplace=True)
        else:
          df1 = df.pop(c)
          df[c] = df1
    return df

In [ ]:
#1. Il cognome e il nome dei poliziotti presenti nel database. Ordinare in ordine alfabetico.
cypher_query = '''
MATCH (o:Officer)
RETURN o.rank,o.surname, o.name ORDER BY(o.surname)
'''

read_cypher(cypher_query)

,o.rank,o.surname,o.name
0,Police Constable,Aberdeen,Tab
1,Inspector,Abramchik,Nomi
2,Police Constable,Ace,Carlota
3,Sergeant,Adin,Dani
4,Police Constable,Adney,Aldric
...,...,...,...
995,Police Constable,Yukhnevich,Alli
996,Police Constable,Yurivtsev,Harold
997,Police Constable,Zamorrano,Viv
998,Sergeant,de Verson,Marylou


In [ ]:
#2. Il cognome e il nome dei poliziotti che hanno il grado (rank) di "Police Constable";
cypher_query = '''
MATCH (o:Officer {rank :'Police Constable'})
RETURN o.surname, o.name 
'''

read_cypher(cypher_query)

,o.surname,o.name
0,Hannam,Christian
1,Clampett,Aguie
2,Death,Von
3,Basley,Isak
4,Aykroyd,Marketa
...,...,...
370,Mayoh,Eloisa
371,Shearme,Hortense
372,Nias,Kassey
373,Emerine,Hertha


In [ ]:
#3. I crimini in cui sono coinvolte BMW modello M o Mercedes-Benz modello M-Class (model)
cypher_query = '''
MATCH (v:Vehicle)-[:INVOLVED_IN]-(c:Crime)
WHERE (v.make = "BMW" AND v.model = "M") OR (v.make = "Mercedes-Benz" AND v.model = "M-Class")
RETURN c
'''

read_cypher(cypher_query)

,date,id,type,last_outcome
0,4/08/2017,a42699bed37958abb8c9096df66a3d533b17540b632141...,Vehicle crime,Investigation complete; no suspect identified
1,28/08/2017,da187294ebd822e6651308e4a2987bbf56e0ab2c9d22d9...,Vehicle crime,Investigation complete; no suspect identified
2,4/08/2017,547b8f2b12aea51b8f0eb229aaa4ec0370f4e67c7d3e1d...,Vehicle crime,Investigation complete; no suspect identified
3,29/08/2017,74dfbc7365739859002b3f3f8e932975e3c64350a564c3...,Vehicle crime,Investigation complete; no suspect identified


In [ ]:
#4. I crimini seguiti dal poliziotto che si chiama Christian Hannam
cypher_query = '''
MATCH (c:Crime) - [inv:INVESTIGATED_BY] - (o:Officer {name: 'Christian' , surname: 'Hannam'})
RETURN c.type
'''

read_cypher(cypher_query)

,c.type
0,Violence and sexual offences
1,Criminal damage and arson
2,Vehicle crime
3,Drugs
4,Criminal damage and arson
5,Vehicle crime
6,Bicycle theft
7,Violence and sexual offences
8,Theft from the person
9,Other theft


In [ ]:
#5. Il tipo dei crimini seguiti da Christian Hannam che sono ancora "under investigation";
cypher_query = '''
MATCH (c:Crime {last_outcome: "Under investigation"}) - [inv:INVESTIGATED_BY] - (o:Officer {name: 'Christian', surname: 'Hannam'})
RETURN c.type
'''

read_cypher(cypher_query)

,date,id,last_outcome,type
0,10/08/2017,e6537cce7e5e701176646e29d4044253189c4cfb51d5ca...,Under investigation,Criminal damage and arson
1,18/08/2017,3ff357e11bda6193e32714678b32cb0fc79df88d493c7e...,Under investigation,Robbery


In [ ]:
#6. Gli indirizzi dei crimini su cui Christian Hannam sta ancora investigando
cypher_query = '''
MATCH (l:Location) - [oc:OCCURRED_AT] - (c:Crime {last_outcome: "Under investigation"}) - [inv:INVESTIGATED_BY] - (o:Officer {name: 'Christian', surname: 'Hannam'})
RETURN l.address
'''

read_cypher(cypher_query)

,l.address
0,13 Stovell Road
1,114 Wigan Road


In [ ]:
#7. L’area dei crimini su cui Christian Hannam sta ancora investigando
cypher_query = '''
MATCH (a:Area) - [loc:LOCATION_IN_AREA] - (l:Location) - [oc:OCCURRED_AT] - (c:Crime {last_outcome: "Under investigation"}) - [inv:INVESTIGATED_BY] - (o:Officer {name: 'Christian', surname: 'Hannam'})
RETURN a.areaCode
'''

read_cypher(cypher_query)

,a.areaCode
0,M40
1,BL3


In [ ]:
#8. Il numero di poliziotti per ogni grado. Ordinare dal grado che ha più poliziotti a scendere.
cypher_query = '''
MATCH (o:Officer)
RETURN o.rank, COUNT(o.rank) as totale ORDER BY(totale) DESC
'''

read_cypher(cypher_query)

,o.rank,totale
0,Police Constable,375
1,Sergeant,336
2,Inspector,199
3,Chief Inspector,90


In [ ]:
#9. Il grado che ha meno di 200 poliziotti
cypher_query = '''
MATCH (o:Officer)
WITH o.rank as r, COUNT(o.rank) as c 
WHERE c < 200
RETURN r, c
'''
# with equivale all'having in sql
read_cypher(cypher_query)

,r,c
0,Inspector,199
1,Chief Inspector,90
